# Análise e Modelagem dos Microdados do ENEM de 20220

Este notebook realiza a **análise exploratória de dados (EDA)** e a **modelagem preditiva** utilizando os microdados do ENEM 2022, utilizando o **PySpark** para processar grandes volumes de dados de forma eficiente.

In [2]:
# !pip install pyspark


Usage:   
  pip3 install [options] <requirement specifier> [package-index-options] ...
  pip3 install [options] -r <requirements file> [package-index-options] ...
  pip3 install [options] [-e] <vcs project url> ...
  pip3 install [options] [-e] <local project path> ...
  pip3 install [options] <archive url/path> ...

no such option: -u



## Importação de Bibliotecas

In [3]:
import pandas as pd
import numpy as np
import pandas as pd
from IPython.core.display import HTML
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql import functions as f
from pyspark.sql.window import Window
from google.colab import drive
from IPython.display import display, HTML


## Criação da Spark Session

A Spark Session  é configurada para garantir um processamento eficiente dos dados, alocando 4GB de memória para as tarefas de análise e modelagem de dados.



In [4]:
# Melhor visualização do DataFrame no notebook
# Isso evita que o conteúdo seja cortado e melhora a legibilidade no output
display(HTML("<style>pre { white-space: pre !important; }</style>"))

# Session builder
spark = SparkSession.builder \
    .appName("Teste SX") \
    .config("spark.executor.memory", "4g") \
    .config("spark.driver.memory", "4g") \
    .getOrCreate()

## Carregamento dos dados

Os dados do ENEM de 2020 são carregados com a inferência automática de tipos de dados e separador adequado. O reparticionamento é aplicado para distribuir melhor as tarefas de processamento no cluster.




In [5]:
# Carregamento dos microdados ENEM 2022 com inferência de schema e separador adequado
df = spark.read.format('csv') \
    .option("sep", ";") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load('/content/DADOS/MICRODADOS_ENEM_2020.csv')

# Otimização das partições para melhorar o desempenho
df_enem = df.repartition(1000)

In [6]:
# Quantidade de linhas do dataset
df_enem.count()

5783109

## Alteração dos valores das colunas do dataset
Realizando a modificação dos valores das colunas do dataset para corresponderem ao dicionário de dados do ENEM 2020.


In [7]:
df_alterado = df \
    .withColumn(
        "TP_FAIXA_ETARIA",
        when(df.TP_FAIXA_ETARIA == 1, "Menor de 17 anos")
        .when(df.TP_FAIXA_ETARIA == 2, "17 anos")
        .when(df.TP_FAIXA_ETARIA == 3, "18 anos")
        .when(df.TP_FAIXA_ETARIA == 4, "19 anos")
        .when(df.TP_FAIXA_ETARIA == 5, "20 anos")
        .when(df.TP_FAIXA_ETARIA == 6, "21 anos")
        .when(df.TP_FAIXA_ETARIA == 7, "22 anos")
        .when(df.TP_FAIXA_ETARIA == 8, "23 anos")
        .when(df.TP_FAIXA_ETARIA == 9, "24 anos")
        .when(df.TP_FAIXA_ETARIA == 10, "25 anos")
        .when(df.TP_FAIXA_ETARIA >= 11, "Entre 26 e 30 anos")
        .when(df.TP_FAIXA_ETARIA >= 12, "Entre 31 e 35 anos")
        .when(df.TP_FAIXA_ETARIA >= 13, "Entre 36 e 40 anos")
        .when(df.TP_FAIXA_ETARIA >= 14, "Entre 41 e 45 anos")
        .when(df.TP_FAIXA_ETARIA >= 15, "Entre 46 e 50 anos")
        .when(df.TP_FAIXA_ETARIA >= 16, "Entre 51 e 55 anos")
        .when(df.TP_FAIXA_ETARIA >= 17, "Entre 56 e 60 anos")
        .when(df.TP_FAIXA_ETARIA >= 18, "Entre 61 e 65 anos")
        .when(df.TP_FAIXA_ETARIA >= 19, "Entre 66 e 70 anos")
        .when(df.TP_FAIXA_ETARIA >= 20, "Maior de 70 anos")
        .otherwise("Desconhecido")
    ) \
    .withColumn(
        "TP_ESTADO_CIVIL",
        when(df.TP_ESTADO_CIVIL == 0, "Não informado")
        .when(df.TP_ESTADO_CIVIL == 1, "Solteiro(a)")
        .when(df.TP_ESTADO_CIVIL == 2, "Casado(a)/Mora com companheiro(a)")
        .when(df.TP_ESTADO_CIVIL == 3, "Divorciado(a)/Desquitado(a)/Separado(a)")
        .when(df.TP_ESTADO_CIVIL == 4, "Viúvo(a)")
        .otherwise("Desconhecido")
    ) \
    .withColumn(
        "TP_COR_RACA",
        when(df.TP_COR_RACA == 0, "Não declarado")
        .when(df.TP_COR_RACA == 1, "Branca")
        .when(df.TP_COR_RACA == 2, "Preta")
        .when(df.TP_COR_RACA == 3, "Parda")
        .when(df.TP_COR_RACA == 4, "Amarela")
        .when(df.TP_COR_RACA == 5, "Indígena")
        .otherwise("Desconhecido")
    ) \
    .withColumn(
        "TP_NACIONALIDADE",
        when(df.TP_NACIONALIDADE == 0, "Não informado")
        .when(df.TP_NACIONALIDADE == 1, "Brasileiro(a)")
        .when(df.TP_NACIONALIDADE == 2, "Brasileiro(a) Naturalizado(a)")
        .when(df.TP_NACIONALIDADE == 3, "Estrangeiro(a)")
        .when(df.TP_NACIONALIDADE == 4, "Brasileiro(a) Nato(a), nascido(a) no exterior")
        .otherwise("Desconhecido")
    ) \
    .withColumn(
        "TP_ST_CONCLUSAO",
        when(df.TP_ST_CONCLUSAO == 1, "Já concluí o Ensino Médio")
        .when(df.TP_ST_CONCLUSAO == 2, "Estou cursando e concluirei o Ensino Médio em 2020")
        .when(df.TP_ST_CONCLUSAO == 3, "Estou cursando e concluirei o Ensino Médio após 2020")
        .when(df.TP_ST_CONCLUSAO == 4, "Não concluí e não estou cursando o Ensino Médio")
        .otherwise("Desconhecido")
    ) \
    .withColumn(
        "TP_ANO_CONCLUIU",
        when(df.TP_ANO_CONCLUIU == 0, "Não informado")
        .when(df.TP_ANO_CONCLUIU == 1, "2019")
        .when(df.TP_ANO_CONCLUIU == 2, "2018")
        .when(df.TP_ANO_CONCLUIU == 3, "2017")
        .when(df.TP_ANO_CONCLUIU == 4, "2016")
        .when(df.TP_ANO_CONCLUIU == 5, "2015")
        .when(df.TP_ANO_CONCLUIU == 6, "2014")
        .when(df.TP_ANO_CONCLUIU == 7, "2013")
        .when(df.TP_ANO_CONCLUIU == 8, "2012")
        .when(df.TP_ANO_CONCLUIU == 9, "2011")
        .when(df.TP_ANO_CONCLUIU >= 10, "2010")
        .when(df.TP_ANO_CONCLUIU >= 11, "2009")
        .when(df.TP_ANO_CONCLUIU >= 12, "2008")
        .when(df.TP_ANO_CONCLUIU >= 13, "2007")
        .when(df.TP_ANO_CONCLUIU >= 14, "Antes de 2007")
        .otherwise("Desconhecido")
    ) \
    .withColumn(
        "TP_ESCOLA",
        when(df.TP_ESCOLA == 1, "Não Respondeu")
        .when(df.TP_ESCOLA == 2, "Pública")
        .when(df.TP_ESCOLA == 3, "Privada")
        .when(df.TP_ESCOLA == 4, "Exterior")
        .otherwise("Desconhecido")
    ) \
    .withColumn(
        "TP_ENSINO",
        when(df.TP_ENSINO == 1, "Ensino Regular")
        .when(df.TP_ENSINO == 2, "Educação Especial - Modalidade Substitutiva")
        .when(df.TP_ENSINO == 3, "Educação de Jovens e Adultos")
    ) \
    .withColumn(
        "IN_TREINEIRO",
        when(df.IN_TREINEIRO == 0, "Não")
        .when(df.IN_TREINEIRO == 1, "Sim")
        .otherwise("Desconhecido")
    ) \
    .withColumn(
        "TP_DEPENDENCIA_ADM_ESC",
        when(df.TP_DEPENDENCIA_ADM_ESC == 1, "Federal")
        .when(df.TP_DEPENDENCIA_ADM_ESC == 2, "Estadual")
        .when(df.TP_DEPENDENCIA_ADM_ESC == 3, "Municipal")
        .when(df.TP_DEPENDENCIA_ADM_ESC == 4, "Privada")
    ) \
    .withColumn(
        "TP_LOCALIZACAO_ESC",
        when(df.TP_LOCALIZACAO_ESC == 1, "Urbana")
        .when(df.TP_LOCALIZACAO_ESC == 2, "Rural")
    ) \
    .withColumn(
        "TP_SIT_FUNC_ESC",
        when(df.TP_SIT_FUNC_ESC == 1, "Em atividade")
        .when(df.TP_SIT_FUNC_ESC == 2, "Paralisada")
        .when(df.TP_SIT_FUNC_ESC == 3, "Extinta")
        .when(df.TP_SIT_FUNC_ESC == 4, "Escola extinta em anos anteriores.")
    ) \
    .withColumn(
        "TP_PRESENCA_CN",
        when(df.TP_PRESENCA_CN == 0, "Faltou à prova")
        .when(df.TP_PRESENCA_CN == 1, "Presente na prova")
        .when(df.TP_PRESENCA_CN == 2, "Eliminado na prova")
        .otherwise("Desconhecido")
    ) \
    .withColumn(
        "TP_PRESENCA_CH",
        when(df.TP_PRESENCA_CH == 0, "Faltou à prova")
        .when(df.TP_PRESENCA_CH == 1, "Presente na prova")
        .when(df.TP_PRESENCA_CH == 2, "Eliminado na prova")
        .otherwise("Desconhecido")
    ) \
    .withColumn(
        "TP_PRESENCA_LC",
        when(df.TP_PRESENCA_LC == 0, "Faltou à prova")
        .when(df.TP_PRESENCA_LC == 1, "Presente na prova")
        .when(df.TP_PRESENCA_LC == 2, "Eliminado na prova")
        .otherwise("Desconhecido")
    ) \
    .withColumn(
        "TP_PRESENCA_MT",
        when(df.TP_PRESENCA_MT == 0, "Faltou à prova")
        .when(df.TP_PRESENCA_MT == 1, "Presente na prova")
        .when(df.TP_PRESENCA_MT == 2, "Eliminado na prova")
        .otherwise("Desconhecido")
    ) \
    .withColumn(
        "TP_STATUS_REDACAO",
        when(df.TP_STATUS_REDACAO == 1, "Sem problemas")
        .when(df.TP_STATUS_REDACAO == 2, "Anulada")
        .when(df.TP_STATUS_REDACAO == 3, "Cópia Texto Motivador")
        .when(df.TP_STATUS_REDACAO == 4, "Em Branco")
        .when(df.TP_STATUS_REDACAO == 6, "Fuga ao tema")
        .when(df.TP_STATUS_REDACAO == 7, "Não atendimento ao tipo textual")
        .when(df.TP_STATUS_REDACAO == 8, "Texto insuficiente")
        .when(df.TP_STATUS_REDACAO == 9, "Parte desconectada")
    )


# Mostra o resultado
df_alterado.show(truncate=False)


+------------+------+------------------+-------+---------------------------------+-----------+-----------------------------+--------------------------------------------------+---------------+-------------+--------------+------------+----------------+-------------------+---------+---------+----------------------+------------------+---------------+------------------+--------------------+-----------+-----------+-----------------+-----------------+-----------------+-----------------+-----------+-----------+-----------+-----------+----------+----------+----------+----------+---------------------------------------------+---------------------------------------------+--------------------------------------------------+---------------------------------------------+---------+---------------------------------------------+---------------------------------------------+--------------------------------------------------+---------------------------------------------+-----------------+-------------+----

## Alteração do nome das colunas
Altera o nome das colunas dos questionarios socioeconomicos para melhor visualização dos dados.

In [8]:
# Lista atual de nomes das colunas do DataFrame
nomes_atuais = df_alterado.columns

# Lista dos novos nomes das colunas que serão atribuídos
novos_nomes = [
    'SERIE_MAX_PAI_HR', 'SERIE_MAX_MAE_MR', 'GRUPO_OCUPACAO_PAI_HR', 'GRUPO_OCUPACAO_MAE_MR',
    'QTD_MORAM_RESIDENCIA', 'RENDA_MENSAL_FAMILIAR', 'QTD_EMPREGADO_DOMESTICO', 'QTD_BANHEIRO',
    'QTD_QUARTOS', 'QTD_CARRO', 'QTD_MOTOCICLETA', 'QTD_GELADEIRA', 'QTD_FREEZER',
    'QTD_MARQUINA_LAVA_ROUPA', 'QTD_MAQUINA_SECAR_ROUPA', 'QTD_MICRO-ONDAS', 'QTD_LAVA_LUCA',
    'QTD_ASPIRADOR', 'QTD_TELEVISAO', 'QTD_DVD', 'QTD_TV_ASSINATURA', 'QTD_TEL_CELULAR',
    'QTD_TEL_FIXO', 'QTD_COMPUTADOR', 'ACESSO_INTERNET'
]

# Lista das colunas atuais que serão alteradas
colunas_para_alterar = [
    'Q001', 'Q002', 'Q003', 'Q004', 'Q005', 'Q006', 'Q007', 'Q008', 'Q009',
    'Q010', 'Q011', 'Q012', 'Q013', 'Q014', 'Q015', 'Q016', 'Q017',
    'Q018', 'Q019', 'Q020', 'Q021', 'Q022', 'Q023', 'Q024', 'Q025'
]

# Criar um dicionário para mapear as colunas antigas para os novos nomes
renomeacao = {colunas_para_alterar[i]: novos_nomes[i] for i in range(len(colunas_para_alterar))}

# Renomear as colunas no DataFrame e converter seus valores para caixa alta
for coluna_antiga, coluna_nova in renomeacao.items():
    # Renomear a coluna antiga para o novo nome
    df_alterado = df_alterado.withColumnRenamed(coluna_antiga, coluna_nova)

# Exibir o DataFrame alterado com as novas colunas e valores em caixa alta
df_alterado.show()


+------------+------+------------------+-------+--------------------+-----------+--------------------+--------------------+---------------+-------------+--------------+------------+----------------+-------------------+---------+---------+----------------------+------------------+---------------+------------------+--------------------+-----------+-----------+-----------------+-----------------+-----------------+-----------------+-----------+-----------+-----------+-----------+----------+----------+----------+----------+--------------------+--------------------+--------------------+--------------------+---------+--------------------+--------------------+--------------------+--------------------+-----------------+-------------+-------------+-------------+-------------+-------------+---------------+----------------+----------------+---------------------+---------------------+--------------------+---------------------+-----------------------+------------+-----------+---------+--------------

## Seleção das colunas mais relevantes

In [9]:
# Lista com as colunas que serão selecionadas
df_cols  = ['NU_INSCRICAO', 'NU_ANO', 'TP_FAIXA_ETARIA', 'TP_SEXO', 'TP_ST_CONCLUSAO','TP_COR_RACA',
               'TP_ESCOLA', 'TP_PRESENCA_CN', 'TP_PRESENCA_CH', 'TP_PRESENCA_LC',
               'TP_PRESENCA_MT', 'CO_PROVA_CN', 'CO_PROVA_CH', 'CO_PROVA_LC',
               'CO_PROVA_MT', 'NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC',
               'NU_NOTA_MT', 'TP_STATUS_REDACAO', 'NU_NOTA_REDACAO',
               'SERIE_MAX_PAI_HR', 'SERIE_MAX_MAE_MR', 'GRUPO_OCUPACAO_PAI_HR',
            'GRUPO_OCUPACAO_MAE_MR','QTD_MORAM_RESIDENCIA', 'RENDA_MENSAL_FAMILIAR']

# Seleciona apenas as colunas importantes
df_reduzido = df_alterado.select(*df_cols)


## Levantando indicadores

- Qual a escola com a maior média de notas?
- Qual o aluno com a maior média de notas e o valor dessa média?
- Qual a média geral?
- Qual o % de Ausentes?
- Qual o número total de Inscritos?
- Qual a média por disciplina?
- Qual a média por Sexo?
- Qual a média por Etnia?

In [10]:
# Cria uma tabela temporária para consultas SQL
df_reduzido.createOrReplaceTempView("ENEM_2020")


In [11]:
# Escola com a maior média de notas
spark.sql("""
SELECT TP_ESCOLA,
       ROUND(AVG((NU_NOTA_CN + NU_NOTA_CH + NU_NOTA_LC + NU_NOTA_MT + NU_NOTA_REDACAO) / 5),2) AS media_total
FROM ENEM_2020
GROUP BY TP_ESCOLA
ORDER BY media_total DESC
LIMIT 1
""").show()


+---------+-----------+
|TP_ESCOLA|media_total|
+---------+-----------+
|  Privada|     605.86|
+---------+-----------+



Obtivemos como resultado que a escola com a maior média de notas é escola Privada.

In [12]:
# Aluno com a maior média de notas e o valor da média
spark.sql("""
SELECT NU_INSCRICAO,
       ROUND(((NU_NOTA_CN + NU_NOTA_CH + NU_NOTA_LC + NU_NOTA_MT + NU_NOTA_REDACAO) / 5), 2) AS media_aluno
FROM ENEM_2020
ORDER BY media_aluno DESC
LIMIT 1
""").show()


+------------+-----------+
|NU_INSCRICAO|media_aluno|
+------------+-----------+
|200005996961|     858.58|
+------------+-----------+



Obtivemos como resultado que o aluno com a maior média foi o aluno com o número de inscrição 200005996961 e sua média foi de 858.58

In [13]:
# Média geral
spark.sql("""
SELECT ROUND(AVG((NU_NOTA_CN + NU_NOTA_CH + NU_NOTA_LC + NU_NOTA_MT + NU_NOTA_REDACAO) / 5),2) AS media_geral
FROM ENEM_2020
""").show()


+-----------+
|media_geral|
+-----------+
|     526.58|
+-----------+



Obtivemos como resultado que a média geral é de 526.58.



In [14]:
# Porcentagem de ausentes
spark.sql("""
SELECT ROUND((COUNT(*) * 100.0 / (SELECT COUNT(*) FROM ENEM_2020)),2) AS porcentagem_ausentes
FROM ENEM_2020
WHERE TP_PRESENCA_CN = 'Faltou à prova' AND TP_PRESENCA_CH = 'Faltou à prova' AND TP_PRESENCA_LC = 'Faltou à prova' AND TP_PRESENCA_MT = 'Faltou à prova'
""").show()


+--------------------+
|porcentagem_ausentes|
+--------------------+
|               52.15|
+--------------------+



Obtivemos como resultado que a porcentagem dos alunos que não compareceram à prova do enem de 2020 foi de 52.15%, mais da metade dos inscritos.

In [15]:
# Número total de inscritos
spark.sql("""
SELECT COUNT(DISTINCT NU_INSCRICAO) AS total_inscritos
FROM ENEM_2020
""").show()

+---------------+
|total_inscritos|
+---------------+
|        5783109|
+---------------+



Obtivemos como resultado que o número total de inscritos foram de 5783109.

In [16]:
# Média por disciplina
spark.sql("""
SELECT ROUND(AVG(NU_NOTA_CN),2) AS media_cn,
       ROUND(AVG(NU_NOTA_CH),2) AS media_ch,
       ROUND(AVG(NU_NOTA_LC),2) AS media_lc,
       ROUND(AVG(NU_NOTA_MT),2) AS media_mt
FROM ENEM_2020
""").show()


+--------+--------+--------+--------+
|media_cn|media_ch|media_lc|media_mt|
+--------+--------+--------+--------+
|  490.41|  511.15|   523.8|  520.58|
+--------+--------+--------+--------+



Obtemos como resultado que a média por disciplina é de 490.41para ciências da natureza, 511.15 para ciências humanas, 523.8 para linguagens e códigos e de 520.58 para matemática. A maior média foi da disciplina de linguagens e códigos

In [17]:
# Média por sexo
spark.sql("""
SELECT TP_SEXO,
       ROUND(AVG((NU_NOTA_CN + NU_NOTA_CH + NU_NOTA_LC + NU_NOTA_MT)/4),2) AS media_por_sexo
FROM ENEM_2020
GROUP BY TP_SEXO
ORDER BY media_por_sexo DESC
""").show()


+-------+--------------+
|TP_SEXO|media_por_sexo|
+-------+--------------+
|      M|        528.58|
|      F|        502.61|
+-------+--------------+



Obtemos como resultado que a média de notas de acordo com o sexo é de 528.58 para masculino e de 502.61 para feminino.

In [18]:
# Média por etnia
spark.sql("""
SELECT TP_COR_RACA,
       ROUND(AVG((NU_NOTA_CN + NU_NOTA_CH + NU_NOTA_LC + NU_NOTA_MT)/4),2) AS media_por_etnia
FROM ENEM_2020
GROUP BY TP_COR_RACA
ORDER BY media_por_etnia DESC
""").show()


+-------------+---------------+
|  TP_COR_RACA|media_por_etnia|
+-------------+---------------+
|       Branca|         539.92|
|Não declarado|         522.08|
|      Amarela|         511.22|
|        Parda|         496.35|
|        Preta|         490.58|
|     Indígena|         467.16|
+-------------+---------------+



Obtemos como resultado que a média de notas de acordo com a etnia é de 539.92 para Branca, 522.08 para não declarado, 511.22 para amarela, 496.35 para parda, 490.58 para Preta e 467.16 para indígena.

## Seleção de dados

Seleção somente dos dados dos candidatos que compareceram à prova.

In [19]:
# Filtrar linhas com notas válidas e status de redação válido
df_reduzido = spark.sql("""
SELECT *
FROM ENEM_2020
WHERE TP_PRESENCA_CN = 'Presente na prova' AND TP_PRESENCA_CH = 'Presente na prova' AND TP_PRESENCA_LC = 'Presente na prova' AND TP_PRESENCA_MT = 'Presente na prova'
""")

## Criação de uma nova coluna

Criação da coluna NOTA_TOTAL visando correlaciona-la outras colunas.

In [20]:
# Cria a coluna de nota total
df_reduzido = df_reduzido.withColumn("NOTA_TOTAL", (df_reduzido.NU_NOTA_CN +
                                                    df_reduzido.NU_NOTA_CH +
                                                    df_reduzido.NU_NOTA_LC +
                                                    df_reduzido.NU_NOTA_MT) / 4)

In [21]:
# Salva o DataFrame como CSV em um único arquivo
# df_reduzido.coalesce(1).write.csv('/content/DADOS_2', header=True, mode='overwrite')
